In [ ]:
#| default_exp config

# EVCI config - for urban areas

> **API**: The inputs for designing and analyzing EVCI sites in urban areas are specified through excel sheets. The python code reads these inputs parameters from the following excel sheets for analysis.

1. model.xlsx - This contains all the global parameters that are model specific. They remain valid for every urban area that is being analyzed.
2. sites.xlsx - This contains a list of sites, with their latitude and longitude for analysis. This is an initial list of sites. This file needs to be created for each urban area. Sites can be filtered based on the planning scenarios.
3. traffic.xlsx - This contains typical traffic profiles for each site category around the sites for each urban area. This file is used to assign a traffic profile to each site based on its category.
4. grid.xlsx - This contains information about neighboring distribution transformers from where the chargers deployed at each site will draw power from.
5. parking.xlsx - This contains multiple worksheets, each corresponding to a parking profile. Each site is assigned a parking profile based on its category.

In [ ]:
#|hide
from nbdev.showdoc import *

# Read input data

In [ ]:
#|export

import os
import numpy as np
import pandas as pd
import json

import geopandas as gpd
import shapely

import warnings
warnings.filterwarnings("ignore")

def check_files_availability(urban_area:str, input_path="data/sites/"):
    "This function simply checks if all the required input files mentioned above are available."

    INPUT_PATH = input_path + urban_area + '/'

    files_not_found = []

    if not os.path.exists(input_path + 'model.xlsx'): files_not_found.append('model.xlsx')
    if not os.path.exists(INPUT_PATH + 'sites.xlsx'): files_not_found.append('sites.xlsx')
    if not os.path.exists(INPUT_PATH + 'traffic.xlsx'): files_not_found.append('traffic.xlsx')
    if not os.path.exists(INPUT_PATH + 'grid.xlsx'): files_not_found.append('grid.xlsx')
    if not os.path.exists(INPUT_PATH + 'parking.xlsx'): files_not_found.append('parking.xlsx')
    
    return files_not_found

`Arguments`:

1. `uban_area`: a string that identifies the urban area being analyzed (e.g. 'goa')
2. `input_path`: a string denoting base directory under which input files (xlsx and others) are available for analysis. Default is `'data/sites/'`

`Returns`:

`files_not_found`: a list of filenames not found in the default and/or specified path.

In [ ]:
#|export

def setup_and_read_data(urban_area:str, input_path="data/sites/", output_path="data/analysis/", request_id=""):
    "This function sets up paths and reads input excel files for a specified corridor"

    INPUT_PATH = input_path + urban_area + '/'
    OUTPUT_PATH = output_path + '/' + request_id + '/'

    if not os.path.exists(OUTPUT_PATH):
        os.mkdir (OUTPUT_PATH)
        
    try:
        model   = pd.read_excel(input_path + "model.xlsx", sheet_name=None)
        sites   = pd.read_excel(INPUT_PATH + "sites.xlsx", sheet_name=None) 
        traffic = pd.read_excel(INPUT_PATH + "traffic.xlsx", sheet_name=None, header=None)
        grid    = pd.read_excel(INPUT_PATH + "grid.xlsx", sheet_name=None)
        parking = pd.read_excel(INPUT_PATH + "parking.xlsx", sheet_name=None, header=None)
    except Exception as e:
        print("error in call setup_and_read_data(): ",e)
    
    return model, sites, traffic, grid, parking, INPUT_PATH, OUTPUT_PATH

`Arguments`:

1. `urban_area`: a string that identifies the urban area being analyzed (e.g. 'goa')
2. `input_path`: a string denoting base directory under which input files (xlsx and other) are available for analysis. Default is `'data/sites/'`
3. `output_path`: a string denoting base directory under which output files will be stored. Default is `'data/analysis/'`
4. `request_id`: a string denoting the request ID (specific to user credentials)

`Returns`:

1. `model`: dataframe of model parameters (from model.xlsx)
2. `sites`: dataframe of sites (from sites.xlsx)
3. `traffic`: dataframe of traffic profiles (from traffic.xlsx)
4. `grid`: dataframe of grid parameters (from grid.xlsx)
5. `parking`: dataframe of parking profiles (from parking.xlsx)
6. `INPUT_PATH`: a string indicating the input_path (e.g. input/goa/)
7. `OUTPUT_PATH`: a string indicating the output path (e.g. output/goa/)


In [ ]:
#example_usage
files_not_fouund = check_files_availability('panaji',input_path='data/sites/')
files_not_fouund

[]

In [ ]:
#example usage
m,s,t,g,p,i,o = setup_and_read_data('panaji', input_path="data/sites/", output_path="data/analysis/")

# Data availability check

Let's check if the input excel sheets provided have the correctly named worksheets within them.

In [ ]:
#| export
def data_availability_check(m,s,t,g,p): 
    "This function checks if the excel files contain the mandatory worksheets."
    
    model_sheets = set(['planning_scenarios','charger_details','chargers_site_categories',
                            'chargers_opportunity_charging', 'battery_specific', 'others'])
    df = s['sites']['Opportunity charging traffic profile']
    traffic_sheets = set(df[df != 0].unique())             
    grid_sheets = set(['grid'])
    parking_sheets = set(s['sites']['Site category'].unique())
    sites_sheets = set(['sites'])
    
    retval = []
    
    if not model_sheets.issubset(set(m.keys())): retval.append('model')
    if not sites_sheets.issubset(set(s.keys())): retval.append('sites')
    if not traffic_sheets.issubset(set(t.keys())): retval.append('traffic')
    if not grid_sheets.issubset(set(g.keys())): retval.append('grid')
    if not parking_sheets.issubset(set(p.keys())): retval.append('parking')
    
    return retval

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)
5. `p`: dataframe of parking parameters (from parking.xlsx)

`Returns`:

A list of xlsx file names wiht missing mandatory sheets

In [ ]:
data_availability_check(m,s,t,g,p)

[]

# Data integrity check

Let's now check if any of the mandatory columns in each of the worksheets are all empty!

In [ ]:
#| export
def data_integrity_check(m,s,t,g,p, verbose=False):
    "This function checks for integrity of excel data by checking missing values."
    missing = []
    
    for x in [m,s,t,g,p]:
        tmpx = {}
        for k in x.keys():
            total = x[k].shape[0]
            tmpx[k] = []
            for c in x[k].columns:
                if sum(pd.isna(x[k][c])) > 0:
                    if verbose: 
                        print(f"Column '{c}' of '{k}' has {sum(pd.isna(x[k][c]))}/{total} missing values")
                    tmpx[k].append(c)
        missing.append(tmpx)
                    
    return missing

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)

`Returns`:

A dictionary of missing columns with their corresponding xlsx filename and worksheet name.

In [ ]:
data_integrity_check(m,s,t,g,p)

[{'planning_scenarios': [],
  'charger_details': [],
  'chargers_site_categories': [],
  'chargers_opportunity_charging': [],
  'battery_specific': [],
  'others': []},
 {'sites': []},
 {'TF1': [], 'TF2': [], 'TF3': []},
 {'grid': []},
 {'BD': [], 'FH': [], 'PP': []}]

In [ ]:
# verbose output
data_integrity_check(m,s,t,g,p,verbose=True)

[{'planning_scenarios': [],
  'charger_details': [],
  'chargers_site_categories': [],
  'chargers_opportunity_charging': [],
  'battery_specific': [],
  'others': []},
 {'sites': []},
 {'TF1': [], 'TF2': [], 'TF3': []},
 {'grid': []},
 {'BD': [], 'FH': [], 'PP': []}]

In [ ]:
#|export

def data_missing_check(sid,file,input_path="data/sites/"):
    """Function checks for missing values in the excel data"""
    try:
        path = input_path+sid+"/"+file
        columns_to_check=[]
        sheet=""
        if file=="sites.xlsx": 
            sheet='sites'
            columns_to_check=["Name","Longitude","Latitude","type of site","Traffic congestion (4 if in city & 2 if on highway)",
            "Year for Site recommendation Hoarding/Kiosk (1 is yes & 0 is no)","Hoarding margin Kiosk margin Available area (in sqm)","Upfront cost per sqm (land)",
            "Yearly cost per sqm (land)","Upfront cost per sqm (kiosk)","Yearly cost per sqm (kiosk)","Upfront cost per sqm (hoarding)",
            "Yearly cost per sqm (hoarding)","Battery swap available (1 is yes and 0 is no)"]
        elif file=="grid.xlsx":
            sheet='grid'
            columns_to_check=["Name of transformer","Address","Longitude","Latitude","Tariff","Power Outage","Available load"]
        elif file=="traffic.xlsx":
            sheet='profile'
            columns_to_check=["Name","vehicles"]

        df=pd.read_excel(path,sheet_name=[sheet])
        tmpx = {}
        for k in df.keys():
            tmpx= []
            for c in df[k].columns:
                if c in columns_to_check:
                    if sum(pd.isna(df[k][c])) > 0:
                        tmpx.append(c)
        if len(tmpx)>0:return {"missing":True,"columns":tmpx}
        else:return {"missing":False}
    except Exception as e:
        print("Error in call data_missing_check(): ",e)


# Read global variables from xlsx

In [ ]:
#|export

def get_category(sid,input_path="data/sites/"):
   """Function fetch site categories available in sites.xlsx file"""
   try:
    path=input_path+sid+"sites.xlsx"
    cols="Site category"
    df=pd.read_excel(path,sheet_name=["sites"])
    unique_=df[cols].unique().to_list()
   except Exception as e:
    print("error in call get_category(): ",e)

In [ ]:
#|export

def get_grid_data(s,g):
    tr_lat = []
    tr_long = []
    tr_di = []
    tr_name = []
    
    try:
        if g['grid'].shape[0] == 0:
            di = [0]*s['sites'].shape[0]
            return di
        
        data = s['sites']
        data['geometry'] = [shapely.geometry.Point(xy) for xy in 
                            zip(data['Longitude'], data['Latitude'])]

        data_df = {}

        data_df = gpd.GeoDataFrame(data, geometry=data['geometry'])
        data_df = data_df.reset_index(drop=True)
        
        s_df = data.copy()
        s_df = s_df.reset_index(drop=True)
        s_df.geometry = data_df.geometry

        data = g['grid']
        data['geometry'] = [shapely.geometry.Point(xy) for xy in 
                            zip(data['Longitude'], data['Latitude'])]

        data_df = {}

        data_df = gpd.GeoDataFrame(data, geometry=data['geometry'])
        data_df = data_df.reset_index(drop=True)
        
        g_df = data.copy()
        g_df = g_df.reset_index(drop=True)
        g_df.geometry = data_df.geometry

        s_crs = gpd.GeoDataFrame(s_df, crs='EPSG:4326')
        s_crs = s_crs.to_crs('EPSG:5234')

        g_crs = gpd.GeoDataFrame(g_df, crs='EPSG:4326')
        g_crs = g_crs.to_crs('EPSG:5234')
        
        for i in range(s_df.shape[0]):
            distance_from_i = g_crs.geometry.distance(s_crs.geometry.loc[i])
            nearest_to_i = distance_from_i.idxmin()
            tr_lat.append(g_df.loc[nearest_to_i]['Latitude'])
            tr_long.append(g_df.loc[nearest_to_i]['Longitude'])
            tr_name.append(g_df.loc[nearest_to_i]['Name of transformer'])
            tr_di.append(distance_from_i[nearest_to_i]/1e3)
    except Exception as e:
        print ("error in call read_grid_data()",e)
    
    s_df['Transformer name'] = tr_name
    s_df['Transformer longitude'] = tr_long
    s_df['Transformer latitude'] = tr_lat
    s_df['Transformer distance'] = tr_di

    return s_df

`Arguments`:

1. `s`: dataframe of sites (from sites.xlsx)
2. `g`: dataframe of grid profile (from grid.xlsx)

`Returns`:

Augmented data frame 's' with lat,long of nearest transformer and its distance in kms from each site

In [ ]:
di = get_grid_data(s,g)
di.head()

,Name,Address,Longitude,Latitude,Site category,No. of charger bundles opportunity charging,No. of charger bundles destination charging,Parking lot size,Peak opportunity charging traffic,Opportunity charging traffic profile,...,Upfront cost per sqm (kiosk),Yearly cost per sqm (kiosk),Upfront cost per sqm (hoarding),Yearly cost per sqm (hoarding),Battery swap available,geometry,Transformer name,Transformer longitude,Transformer latitude,Transformer distance
0,Miramar Beach,North Beach Parking,73.808845,15.483450,PP,1,1,0,70,TF1,...,0,135000,0,0,0,POINT (73.80885 15.48345),CLUBE TENNIS DE GASPER,73.809155,15.482570,0.103548
1,Caculo Mall,Malls,73.817859,15.486461,PP,1,1,0,70,TF1,...,0,135000,0,0,0,POINT (73.81786 15.48646),CACULO MALL,73.818163,15.486739,0.045180
2,Panjim KTC Bus Stand,Transit locations - Bus stands,73.838266,15.496108,PP,1,1,0,70,TF1,...,0,135000,0,0,0,POINT (73.83827 15.49611),KTC Bus stand,73.838134,15.495092,0.114134
3,Multi-Level Car Park GTDC,CESL Proposed Panaji locatio,73.836559,15.499065,PP,1,1,0,70,TF1,...,0,135000,0,0,0,POINT (73.83656 15.49906),Managing Director(GTDC),73.836074,15.499388,0.063610
4,INOX Goa,CESL Proposed Panaji locatio,73.821097,15.498726,PP,1,1,0,70,TF1,...,0,135000,0,0,0,POINT (73.82110 15.49873),INOX LEISURE LTD (MULTIPLEX),73.821407,15.498623,0.035425


In [ ]:
#|export

def read_globals(m,s,t,g,p,charging_type,ui_inputs):
  "This function returns all global parameters read from the xlsx."
  
  x = json.dumps(ui_inputs)
  ui_inputs = json.loads(x)

  r = {}

  df_p = m['planning_scenarios']
  df_c = m['charger_details']
  df_sc = m['chargers_site_categories']
  df_b = m['battery_specific']
  df_o = m['others']
  
  scenario_code = df_p[df_p['Site categories']==ui_inputs['planning_scenario']]['Scenario code'].iloc[0]

  # read all other parameters from the xlsx
  
  r['scenario_code']=scenario_code
  r['M'] = df_p[df_p['Site categories']==ui_inputs['planning_scenario']]['Charger types'].iloc[0].split(',')
  r['C'] = r['M']

  r['Kj'] = {}
  r['Dj'] = {}
  r['Hj'] = {}
  r['Qj'] = {}
  r['tj'] = {}
  r['Mj'] = {}
  r['Gk'] = {}
  r['Cij'] = {}

  for c in r['C']:
    df_t = df_c[df_c['Type of vehicle']==c]
    charger = df_t['Compatible charger'].iloc[0]
    if charging_type == 'opportunity_charging':
      r['Cij'][c] = [i*df_sc[(df_sc['Chargers']==charger) & (df_sc['Site categories']==ui_inputs['planning_scenario'])]['No. of chargers'].iloc[0] for i in s['sites']['No. of charger bundles opportunity charging']]
    else:
      r['Cij'][c] = [i*df_sc[(df_sc['Chargers']==charger) & (df_sc['Site categories']==ui_inputs['planning_scenario'])]['No. of chargers'].iloc[0] for i in s['sites']['No. of charger bundles destination charging']]
    r['Kj'][c] = int(df_t['Capex per charger'].iloc[0].split('-')[0])
    r['Dj'][c] = df_t['Charging power'].iloc[0]
    #r['Hj'][c] = df_t['Required space per charger'].iloc[0]
    r['Qj'][c] = df_t['Annual maintenance per charger'].iloc[0]
    if charging_type == "opportunity_charging":
      r['tj'][c] = df_t['Charging time for opportunity charging (hrs)'].iloc[0]
    else:
      r['tj'][c] = df_t['Charging time for destination charging (hrs)'].iloc[0]

  r['N'] = 500
  r['Ng'] = 0

  r['timeslots'] = {k: 24/v for k, v in r['tj'].items()}
  timeslots = r['timeslots']
  
  df_t = s['sites']
  r['Nc'] = df_t[df_t['Site category']==scenario_code].shape[0]
  Nc = r['Nc']

  r['Gi'] = [0]*Nc
  r['Ri'] = [0]*Nc
  r['Wi'] = [ui_inputs['cabling_cost']]*Nc
  r['Ai'] = [ui_inputs['Ai']]*Nc
  r['Li'] = [ui_inputs['Li']]*Nc
  r['Bi'] = [ui_inputs['Bipc'] * ui_inputs['Birate'] * 24 * 365]*Nc # e.g. 25% of Rs 3.5/KWh per year

  r['MH'] = [s['sites'].loc[i]['Hoarding margin'] for i in range(Nc)]
  r['MK'] = [s['sites'].loc[i]['Kiosk margin'] for i in range(Nc)]

  r['Eg'] = {k: [ui_inputs['Eg']] * int(v) for k, v in timeslots.items()}
  r['Er'] = {k: [0] * int(v) for k, v in timeslots.items()}
  r['Mg'] = {k: [ui_inputs['Eg'] * r['MK'][0]] * int(v) for k, v in timeslots.items()} # FIX THIS index 0 !!
  r['Mr'] = {k: [0] * int(v) for k, v in timeslots.items()}
  r['l']  = {k: [1] * int(v) for k, v in timeslots.items()}
    
  r['K'] = ui_inputs['years_of_analysis']
  r['charger_types'] = r['M']
  r['years_of_analysis'] = ui_inputs['years_of_analysis']
  r['capex_2W']  = ui_inputs['capex_2W']
  r['capex_3WS'] = ui_inputs['capex_3WS']
  r['capex_4WS'] = ui_inputs['capex_4WS']
  r['capex_4WF'] = ui_inputs['capex_4WF']
  r['hoarding_cost'] = 900000
  r['kiosk_cost'] = 180000
  r['year1_conversion'] = ui_inputs['year1_conversion']
  r['year2_conversion'] = ui_inputs['year2_conversion']
  r['year3_conversion'] = ui_inputs['year3_conversion']
  r['fast_charging'] = ui_inputs['fast_charging']
  r['slow_charging'] = ui_inputs['slow_charging']
  r['holiday_percentage'] = ui_inputs['holiday_percentage']
  
  # now lets derive all other parameters that depend on the UI inputs.
  r['CH'] = [r['hoarding_cost']]*Nc
  r['CK'] = [r['kiosk_cost']]*Nc
  r['pj'] = {1: r['year1_conversion'], 
        2: r['year2_conversion'], 
        3: r['year3_conversion']}

  r['Pj'] = max(r['pj'].values()) 

  #Traffic profile/ Parking profile
  # read hourly vehicular traffic from the traffic.xlsx or parking.xlsx depending on charging_type
  
  p_df = {'opportunity_charging': t, 'destination_charging': p}

  profiles = list(p_df[charging_type].keys())
  djworking = {}

  vehicle_type = {
    "2W": "2W",
    "3W": "3W",
    "4WS": "4W",
    "4WF": "4W",
    "Bus": "Bus",
  }

  for profile in profiles:
    avg_traffic = p_df[charging_type][profile].iloc[3:27,1].to_list()
    avg_traffic_per_type = {}
    for c in r['M']:
      tmp_df = p_df[charging_type][profile]
      frac = tmp_df[tmp_df[0]==vehicle_type[c]].iloc[0,1]
      avg_traffic_per_type[c] = [i*frac for i in avg_traffic]
      # stretch or compress here based on timeslots
      if r['timeslots'][c] > 24:
        scale = int(r['timeslots'][c]/24)
        avg_traffic_per_type[c] = [i for i in avg_traffic_per_type[c] for _ in range(scale)]
      else:
        scale = int(24/r['timeslots'][c])
        avg_traffic_per_type[c] = avg_traffic_per_type[c][::scale]
      djworking[c] = [np.round(i,2) for i in avg_traffic_per_type[c]]
    r['djworking'] = djworking

    djholiday = {}
    for profile in profiles:
      tmp_df = p_df[charging_type][profile]
      holiday_percentage = r['holiday_percentage'] = tmp_df[tmp_df[0]=='holiday_percentage'].iloc[0,1]
      for c in r['M']:
        djholiday[c] = [np.round(i*holiday_percentage,2) for i in djworking[c]]
    r['djholiday'] = djholiday
    
    for profile in profiles:
      tmp_df = p_df[charging_type][profile]
      fast_charging = tmp_df[tmp_df[0]=='fast_charging'].iloc[0,1]
      slow_charging = tmp_df[tmp_df[0]=='slow_charging'].iloc[0,1]
      r['qjworking'] = {}
      r['qjholiday'] = {}
      for c in r['M']:
        r['qjworking'][c] = [slow_charging + fast_charging] * int(timeslots[c])
        r['qjholiday'][c] = [slow_charging + fast_charging] * int(timeslots[c])
    
  return r

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)
5. `p`: dataframe of parking parameters (from parking.xlsx)
6. `charging_type`: a string `opportunity_charging` or `destination_charging`
7. `ui_inputs`: dictionary of all parameters selected by use from the frontend with the UI

`Returns`:

A dictionary with all hyperparameters required for the model to run

In [ ]:
# example usage

ui_inputs = { 
    "planning_scenario": "Public places",
    "years_of_analysis": 2,
    "Ai": 50,
    "Li": 1500,
    "Bipc": .25,
    "Birate": 3.5,
    "MK": .15,
    "Eg": 5.5,
    "cabling_cost":500,
    "capex_2W": 2500,
    "capex_3WS": 112000,
    "capex_4WS": 250000,
    "capex_4WF": 1500000,
    "hoarding cost": 900000,
    "kiosk_cost": 180000,
    "year1_conversion": 0.02,
    "year2_conversion": 0.05,
    "year3_conversion": 0.1,
    "holiday_percentage": 0.3,
    "fast_charging": 0.3,
    "slow_charging": 0.15,
}

r = read_globals(m,s,t,g,p, 'opportunity_charging', ui_inputs)
print(r['Kj'])
print(r['charger_types'])
print(r['Cij']['2W'])

{'2W': 40000, '4WS': 200000, '4WF': 1825000}
['2W', '4WS', '4WF']
[3, 3, 3, 3, 3, 3, 3, 3, 3]
